In [6]:
import librosa
import numpy as np
import tensorflow as tf
import pickle
import os

# === Load Model & Label Encoder ===
model = tf.keras.models.load_model("emotion_gru_attention_model.h5", compile=False)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# === Prediction Function ===
def predict_emotion(audio_path, n_mfcc=40, max_pad_len=174):
    if not os.path.exists(audio_path):
        print("❌ File not found:", audio_path)
        return None

    try:
        y, sr = librosa.load(audio_path, res_type='kaiser_fast')
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

        if mfcc.shape[1] < max_pad_len:
            pad_width = max_pad_len - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_pad_len]

        mfcc = np.transpose(mfcc[np.newaxis, ...], (0, 2, 1))

        prediction = model.predict(mfcc)
        pred_index = np.argmax(prediction)
        pred_label = label_encoder.inverse_transform([pred_index])[0]

        print(f"🎤 Predicted Emotion: **{pred_label}**")
        return pred_label

    except Exception as e:
        print("❌ Error loading audio:", str(e))
        return None


In [12]:
# Example path to your audio file
test_file = "Ravdess/Actor_01/03-02-06-02-02-02-01.wav"  # change this to your test file

predict_emotion(test_file)


1/1 [==============================] - 0s 42ms/step
🎤 Predicted Emotion: **fearful**


'fearful'